In [6]:
import numpy as np
import pandas as pd
import pvlib
from pvlib import solarposition

In [3]:
data = pd.read_csv('8871462_-22.06_-48.17_2019.csv', header = 2)
data['datetime'] = pd.to_datetime(data[['Year','Month', 'Day', 'Hour', 'Minute']])
data['datetime_utc'] = data['datetime'].dt.tz_localize('UTC')
data['datetime_local'] = data['datetime_utc'].dt.tz_convert('America/Sao_Paulo') # Fuso -2 até 16/02 (fim do horário de verão)
data['datetime_local'] = pd.to_datetime(data['datetime_local'])
display(data)

,Year,Month,Day,Hour,Minute,DHI,Temperature,Wind Speed,Solar Zenith Angle,Surface Albedo,GHI,DNI,datetime,datetime_utc,datetime_local
0,2019,1,1,0,0,0,21.6,0.9,134.75,0.17,0,0,2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2018-12-31 22:00:00-02:00
1,2019,1,1,0,30,0,21.3,0.8,134.78,0.17,0,0,2019-01-01 00:30:00,2019-01-01 00:30:00+00:00,2018-12-31 22:30:00-02:00
2,2019,1,1,1,0,0,21.0,0.7,133.66,0.17,0,0,2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2018-12-31 23:00:00-02:00
3,2019,1,1,1,30,0,20.9,0.6,131.45,0.17,0,0,2019-01-01 01:30:00,2019-01-01 01:30:00+00:00,2018-12-31 23:30:00-02:00
4,2019,1,1,2,0,0,20.7,0.6,128.30,0.17,0,0,2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2019-01-01 00:00:00-02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,31,21,30,0,23.4,0.9,119.48,0.17,0,0,2019-12-31 21:30:00,2019-12-31 21:30:00+00:00,2019-12-31 18:30:00-03:00
17516,2019,12,31,22,0,0,23.1,0.9,124.08,0.17,0,0,2019-12-31 22:00:00,2019-12-31 22:00:00+00:00,2019-12-31 19:00:00-03:00
17517,2019,12,31,22,30,0,22.8,1.0,128.06,0.17,0,0,2019-12-31 22:30:00,2019-12-31 22:30:00+00:00,2019-12-31 19:30:00-03:00
17518,2019,12,31,23,0,0,22.5,1.0,131.27,0.17,0,0,2019-12-31 23:00:00,2019-12-31 23:00:00+00:00,2019-12-31 20:00:00-03:00


Kt = G0/GHI 

G0 = Irradiância Extraterrestre

GHI = Irradiância Global Horizontal

In [ ]:
solpos = solarposition.get_solarposition(data['datetime_utc'], latitude, longitude) 

In [31]:
latitude = -23.55052
longitude = -46.633308
altitude = 590
albedo = 0.08 
solpos = solarposition.get_solarposition(data['datetime_utc'], latitude, longitude) 
display(solpos)
zenith = solpos['zenith']
zenith = zenith.values
zenith = pd.Series(zenith).reset_index(drop=True)
display(zenith)


,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
datetime_utc,,,,,,
2019-01-01 00:00:00+00:00,114.476095,114.476095,-24.476095,-24.476095,228.129189,-3.195874
2019-01-01 00:30:00+00:00,119.374683,119.374683,-29.374683,-29.374683,222.677718,-3.205782
2019-01-01 01:00:00+00:00,123.756554,123.756554,-33.756554,-33.756554,216.419942,-3.215687
2019-01-01 01:30:00+00:00,127.491193,127.491193,-37.491193,-37.491193,209.279223,-3.225590
2019-01-01 02:00:00+00:00,130.431269,130.431269,-40.431269,-40.431269,201.247715,-3.235491
...,...,...,...,...,...,...
2019-12-31 21:30:00+00:00,85.161122,85.330101,4.838878,4.669899,246.845935,-3.032026
2019-12-31 22:00:00+00:00,91.579861,91.579861,-1.579861,-1.579861,243.925505,-3.041970
2019-12-31 22:30:00+00:00,97.666672,97.666672,-7.666672,-7.666672,240.669099,-3.051912


0        114.476095
1        119.374683
2        123.756554
3        127.491193
4        130.431269
            ...    
17515     85.330101
17516     91.579861
17517     97.666672
17518    103.550002
17519    109.177629
Length: 17520, dtype: float64

In [25]:
zenith = data['Solar Zenith Angle']
display(zenith)

0        134.75
1        134.78
2        133.66
3        131.45
4        128.30
          ...  
17515    119.48
17516    124.08
17517    128.06
17518    131.27
17519    133.55
Name: Solar Zenith Angle, Length: 17520, dtype: float64

In [32]:
dni_extra = []
for i, row in data.iterrows():
    dniextra = pvlib.irradiance.get_extra_radiation(row['datetime_utc'])
    dni_extra.append(dniextra)

display(dni_extra)

[1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1413.981805,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167304782,
 1414.0080167

In [33]:
# Irradiância extraterrestre 
g0 = np.zeros_like(zenith) #Inicializa um array só de zeros no mesmo formato do array zenith

valid_indices = zenith[zenith <= 90].index #Pega os indices cujos valores de zenite <= a 90 (período diurno)
print(valid_indices)
for index in valid_indices:
    g0[index] = dni_extra[index]*np.cos(np.radians(zenith[index])) #Calcula g0 para os indices válidos

g0[g0 == 0] = np.nan #Substitui zero por nan
data['Kt'] = data['GHI'] / g0
data['date'] = data['datetime_local'].dt.date
daily_kt = data.groupby('date')['Kt'].mean() #Média diária do indice kt

Index([   17,    18,    19,    20,    21,    22,    23,    24,    25,    26,
       ...
       17506, 17507, 17508, 17509, 17510, 17511, 17512, 17513, 17514, 17515],
      dtype='int64', length=8736)


In [36]:
data.to_csv('teste.csv')

In [37]:
display(daily_kt)

date
2018-12-31         NaN
2019-01-01    0.540163
2019-01-02    2.314688
2019-01-03    3.564965
2019-01-04    2.161083
                ...   
2019-12-27    1.489879
2019-12-28    1.508588
2019-12-29    1.676314
2019-12-30    1.803986
2019-12-31    1.900205
Name: Kt, Length: 366, dtype: float64